In [28]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, save_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
import keras
# from keras.applications.xception import Xception
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.mobilenetv2 import MobileNetV2
# from keras.applications.densenet import DenseNet121
import cv2
import os
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
# from keras.preprocessing.image import load_img, img_to_array


import warnings
warnings.filterwarnings('ignore')

def display_image(image, title=''):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Reading images

In [2]:
#vgg can't take input of 28x28 so we shall resize all our images
pickle_folder = "pickle files"
input_shape=(84, 84, 1)
data = pickle.load(open('data.pickle', 'rb'))
# print("Data of format:")
# for ch in data:
#     print("data_length(" + ch + ") =", len(data[ch]))

#no. of classes
classes = len(data.keys())
# print("\n\nClasses:", classes)

# Groups of most misclassified characters

In [17]:
groups =    [
    
    ['उ', 'ऊ', ],
    ['अ', 'ऊ', ],
    ['स', 'ए', 'र', ],
    ['क', 'झ',],
    ['श', 'ग', ],
    ['घ', 'ध', ],
    ['ङ', 'ड',],
    ['ज', 'ञ', 'ज्ञ',],
    ['ठ', 'ट', 'ढ', ],
    ['ढ', 'द',],
    ['भ', 'म', 'क्ष'],
    ['थ', 'य', ],
    ['ब', 'व'],

]

# CNN

In [4]:
import tensorflow as tf
from keras import backend as k
 
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))


print("GPUs:", k.tensorflow_backend._get_available_gpus())

print("input shape:", input_shape)
print("classes:", classes)

def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

GPUs: ['/job:localhost/replica:0/task:0/device:GPU:0']
input shape: (84, 84, 1)
classes: 41


In [30]:
%%time

batch_size = 32
num_classes = classes
epochs = 50
k_r = 0.01
b_r = 0.01
dropout = 0.5

def train_CNN(X_train, y_train):
    early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=0, mode='auto')

    model = Sequential()
    #conv layer 
    model.add(Conv2D(32, kernel_size=(9, 9),
                     activation='relu',
                      kernel_initializer='he_normal',
                      kernel_regularizer=keras.regularizers.l2(k_r),
                      bias_regularizer=keras.regularizers.l2(b_r),
                     input_shape=input_shape))
    model.add(MaxPooling2D((3, 3)))
    # model.add(Dropout(0.05))
    # model.add(Conv2D(32, kernel_size=(3, 3),
    #                  activation='relu',
    #                   #kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(0.1),
    #                   bias_regularizer=keras.regularizers.l2(0.05)))
    # model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropout))

    # model.add(Conv2D(64, kernel_size=(3, 3),
    #                  activation='relu',
    #                   kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(k_r),
    #                   bias_regularizer=keras.regularizers.l2(b_r)))
#     model.add(Conv2D(64, kernel_size=(3, 3),
#                      activation='relu',
#                       kernel_initializer='he_normal',
#                       kernel_regularizer=keras.regularizers.l2(k_r),
#                       bias_regularizer=keras.regularizers.l2(b_r)))
#     model.add(MaxPooling2D((2, 2)))
#     model.add(Dropout(dropout))

    # model.add(Conv2D(128, kernel_size=(3, 3),
    #                  activation='relu',
    #                   kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(k_r),
    #                   bias_regularizer=keras.regularizers.l2(b_r)))
    # model.add(Conv2D(128, kernel_size=(3, 3),
    #                  activation='relu',
    #                   kernel_initializer='he_normal',
    #                   kernel_regularizer=keras.regularizers.l2(k_r),
    #                   bias_regularizer=keras.regularizers.l2(b_r)))
    # model.add(MaxPooling2D((2, 2)))
    # model.add(Dropout(dropout))

    # NN
    model.add(Flatten())
    #hidden layer 1
    # model.add(Dense(1024, activation='relu'))
    # model.add(Dropout(0.05))
    #hidden layer 2
    model.add(Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(k_r)))
    model.add(Dropout(dropout))
    #output layer
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    print(model.summary())
    print("\n############################")
    print("## Memory usage:", get_model_memory_usage(batch_size=batch_size, model=model), "GB ##")
    print("############################\n")

    history = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.15,
            callbacks = [early_stopping_callback])
    return model
# score = model.evaluate(X_test, y_test, verbose=0)
# print(score)

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 13.8 µs


# Producing models for different groups

In [6]:
file = "class_numerals_{}.pickle".format(classes)
if file in os.listdir(pickle_folder):
    class_numerals = pickle.load(open(os.path.join(pickle_folder, file), 'rb'))
    print("Unpickled")
else:
    class_numerals = {i:list(data.keys())[i] for i in range(classes)}
    pickle.dump(class_numerals, open(os.path.join(pickle_folder, file), 'wb'))
    print("Error. Incompatible. Pickled new file:", file)
def character_to_encoding(char, classes=classes):
    for i in class_numerals:
        if class_numerals[i] == char:
            return np_utils.to_categorical(i, classes)

def encoding_to_character(encoding):
    return class_numerals[encoding.argmax()]

Unpickled


In [ ]:
X = []
y = []
write_folder = "pickle files/group_CNNs/"

for group in groups:
    #for each group
    X = []
    y = []
    
    #creating our dataset
    for ch in group:
        X += data[ch]
        y += [ch] * len(data[ch])
        
    #resizing input image and preprocessin
    X = np.array(list(map(lambda x: cv2.resize(x/255, (input_shape[0], input_shape[1])).reshape((input_shape[0], input_shape[1], 1)), X)))
    y = np.array(list(map(lambda x: character_to_encoding(x, classes), y)))

    # Train-Test split

    train_ratio = 0.8
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=20, shuffle=True, stratify=y)

    print(group)
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
#     print("\n\n")
    
    model = train_CNN(X_train, y_train)
#     accuracy = model.evaluate(X_test, y_test)
    y_test_classed = list(map(encoding_to_character, y_test))
    predictions = list(map(encoding_to_character, model.predict(X_test)))
    accuracy = accuracy_score(y_pred=predictions, y_true=y_test_classed)
    print("Accuracy =", accuracy)
    print(confusion_matrix(y_pred=predictions, y_true=y_test_classed))
    file = write_folder+'-'.join(group)+'_'+str(accuracy)[:6]+'.h5'
    if os.path.exists(file):
        os.remove(file)
#     keras.models.save_model(model, file)
#     print("Wrote model at", file)
#     print("\n\n\n\n")